In [1]:
import sys
import os
sys.path.insert(0, '../lib/')

# help_functions.py
from image_processing import *
from pre_processing import *
from my_model import *


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Loading Data

In [2]:
data_path = "/home/ec2-user/data/IDRiD/hdf5/"

train_imgs = load_hdf5(data_path + "loc_train_imgs")
train_center = load_hdf5(data_path + "loc_train_center")

test_imgs = load_hdf5(data_path + "loc_test_imgs")
test_imgs = load_hdf5(data_path + "loc_test_center")

# Parameters Setting

In [38]:
# Create the experiment folder
experiment= "IDRiD_localizer"
result_dir = "/home/ec2-user/od-segmentation/results/" + experiment + "/"
exp_number = 1

# If this is the first experiement, create a new folder
if not os.path.exists(result_dir):
    os.mkdir(model_path)
# If this is not the first experiement, set the experiment number to be the last experiment + 1
else:
    exp_dir = os.listdir(result_dir)
    last_exp = exp_dir[-1]
    if not last_exp == '.ipynb_checkpoints':
        exp_number = int(last_exp.split("_")[-1])
    
result_path = result_dir + "exp_" + str(exp_number) + '/'
if not os.path.exists(result_path):
    os.mkdir(result_path)

#training settings
N_epochs = 1
batch_size = 1
optimizer = 'rmsprop'
metrics = ['mse']
learning_rate = 0.01

# Write Log File
with open(result_path + 'configuration.txt','w') as out:
    line0 = experiment + ': exp_' + str(exp_number)
    line1 = 'N_epochs = ' + str(N_epochs)
    line2 = 'batch_size = ' + str(batch_size)
    line3 = 'optimizer = ' + optimizer
    line4 = 'metrics = ' + metrics[0]
    line5 = 'learning rate = ' + str(learning_rate)
    out.write('{}\n{}\n{}\n{}\n{}\n{}\n'.format(line0,line1,line2,line3,line4,line5)) 

In [12]:
# Image Preprocessing function
t = np.swapaxes(train_imgs, 1, 3)
X_train = my_PreProc(t)
y_train = train_center


# Training

In [13]:
# Model
localizer = get_unet_loc2(1,480,480)
localizer.compile(loss='mean_squared_error', optimizer= optimizer, metrics= metrics)

checkpointer = ModelCheckpoint(result_path + "best_weights.h5", verbose=1, monitor='val_loss', mode='auto', save_best_only=True) #save at each epoch if the validation decrease



../lib/my_model.py:621: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  model = Model(input=inputs, output=conv6)


In [14]:
# Fit
history = localizer.fit(X_train, y_train, epochs = N_epochs, batch_size=batch_size, verbose=1, shuffle=True, validation_split=0.1, callbacks=[checkpointer])

Train on 371 samples, validate on 42 samples
Epoch 1/1
  2/371 [..............................] - ETA: 1:18:36 - loss: 84940.3818 - mean_squared_error: 84940.3799

KeyboardInterrupt: 

In [15]:
model_path

'/home/ec2-user/od-segmentation/results/IDRiD_localizer/'